In [7]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw

mesh = ngs.Mesh('whatever.vol')
mesh.Curve(2)
# mesh.Refine()

print("using 3d mesh with ne=", mesh.ne, " elements and nv=", mesh.nv, " points")

deg = 2

using 3d mesh with ne= 70591  elements and nv= 12275  points


In [8]:
HcurlCoil = ngs.HCurl(mesh, order = deg, dirichlet = "coil_outer|coil_inner|coil_up|coil_down", definedon = 'coil', nograds = True)
TotalCurrent = 1000
coil_depth = 0.1

n = ngs.specialcf.normal(mesh.dim)
gfuDirichlet = ngs.GridFunction(HcurlCoil)
g = ngs.CF([(0,0,TotalCurrent/coil_depth) if bc=="coil_inner" else (0,0,0) for bc in mesh.GetBoundaries()])
gfuDirichlet.Set(g,ngs.BND)

# Draw(gfuDirichlet, mesh, vectors = { "grid_size":200}, draw_vol=False, clipping={"z": -1, "dist": 0.064});

In [9]:
from ngsolve.krylovspace import CGSolver

# Weak form
T, Tstar = HcurlCoil.TnT()
Kt = ngs.BilinearForm(ngs.curl(Tstar)*ngs.curl(T)*ngs.dx)
    
# Assembly
with ngs.TaskManager(): Kt.Assemble()

r = - Kt.mat * gfuDirichlet.vec

# Solving
maxres = 1e-5
maxit = 1000
                              
Tsol = ngs.GridFunction(HcurlCoil)
print("Solving...")

with ngs.TaskManager():
    iterativeSolver = CGSolver(Kt.mat, freedofs = HcurlCoil.FreeDofs(), atol  = maxres,  maxiter = maxit)
    Tsol.vec.data = iterativeSolver * r + gfuDirichlet.vec

# if len(iterativeSolver.residuals) == maxit: print("... Failure!")
# else: print("... Success!")
print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")

# Draw(ngs.curl(Tsol), mesh, vectors = { "grid_size" : 150},clipping = {"x" : 0, "y" : 0, "z" : -1, "dist" : 0})
with ngs.TaskManager():
    flux_bd = ngs.Integrate(ngs.curl(Tsol) * n, mesh, definedon = mesh.Boundaries("coil_cut_2"))
print(flux_bd)

Solving...
Number of iterations = 186/1000 | Residual = 9.265082957271972e-06
2999.999999999988


In [10]:
# Hcurl = ngs.HCurl(mesh, order = p, nograds = True)
Hcurl = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'ambient_face')
u,v = Hcurl.TnT()

K = ngs.BilinearForm(ngs.curl(u)*ngs.curl(v)*ngs.dx)
c = ngs.Preconditioner(K, type = "local")

with ngs.TaskManager(): K.Assemble()

f = ngs.LinearForm(Hcurl)
f +=  ngs.curl(Tsol)* ngs.curl(v) * ngs.dx

with ngs.TaskManager(): f.Assemble()

Hs = ngs.GridFunction(Hcurl)
print("Solving...")

with ngs.TaskManager():
    iterativeSolver = CGSolver(K.mat, c.mat, atol  = maxres,  maxiter = maxit)
    # iterativeSolver = CGSolver(K.mat, freedofs = Hcurl.FreeDofs(), tol  = maxres,  maxiter = maxit)
    Hs.vec.data = iterativeSolver * f.vec

# if len(iterativeSolver.residuals) == maxit: print("... Failure!")
# else: print("... Success!")
print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")

# Draw(ngs.curl(Hs), mesh, vectors = { "grid_size" : 150},clipping = {"x" : 0, "y" : 0, "z" : -1, "dist" : 0})

with ngs.TaskManager():
    flux_bd = ngs.Integrate(ngs.curl(Hs) * n, mesh, definedon = mesh.Boundaries("coil_cut_2"))
print(flux_bd)

Solving...
Number of iterations = 284/1000 | Residual = 9.487836146455303e-06
3000.6006312808977


In [11]:
# Draw(ngs.curl(Hs), mesh, vectors = { "grid_size" : 500},clipping = {"x" : 0, "y" : 0, "z" : -1, "dist" : 0})